In [1]:
from pyspark.sql import SparkSession
import logging

In [2]:
%reload_kedro

                    INFO     Resolved project path as: C:\GP\music-listening-behaviour.             __init__.py:139
                             To set a different path, run '%reload_kedro <project_root>'                           

[08/30/23 12:23:16] WARNING  C:\Program                                                             warnings.py:109
                             Files\Python310\lib\site-packages\kedro\framework\session\session.py:2                
                             66: FutureWarning: ConfigLoader will be deprecated in Kedro 0.19.                     
                             Please use the OmegaConfigLoader instead. To consult the documentation                
                             for OmegaConfigLoader, see here:                                                      
                             https://docs.kedro.org/en/stable/configuration/advanced_configuration.                
                             html#omegaconfigloader                                                                
                               warnings.warn(                                                                      
                                                                                                                   

[08/30/23 12:23:17] INFO     Kedro project music-listening-behaviour                                __init__.py:108

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:109
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        __init__.py:115

In [4]:
# check that the spark hook was triggered and that the session is live
SparkSession.builder.getOrCreate() # OK

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.4.1 
 Master 
 local[6] 
 AppName 
 music-listening-behaviour

In [3]:
spark = SparkSession.builder.getOrCreate()
assert spark.conf.get("spark.app.name") == 'music-listening-behaviour'

In [15]:
tracksess = catalog.load('track-sessions')

[08/29/23 21:17:38] INFO     Loading data from 'track-sessions' (SparkDataSet)...               data_catalog.py:475

In [5]:
tracksess.columns

['userid', 'timestamp', 'artistid', 'artistname', 'traid', 'traname']

In [6]:
tracksess.show(100)

+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     userid|           timestamp|            artistid|          artistname|               traid|             traname|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|user_000001|2009-05-04T23:08:57Z|f1b1cf71-bd35-4e9...|           Deep Dish|                null|Fuck Me Im Famous...|
|user_000001|2009-05-04T13:54:10Z|a7f7df4a-77d8-4f1...|            坂本龍一|                null|Composition 0919 ...|
|user_000001|2009-05-04T13:52:04Z|a7f7df4a-77d8-4f1...|            坂本龍一|                null|Mc2 (Live_2009_4_15)|
|user_000001|2009-05-04T13:42:52Z|a7f7df4a-77d8-4f1...|            坂本龍一|                null|Hibari (Live_2009...|
|user_000001|2009-05-04T13:42:11Z|a7f7df4a-77d8-4f1...|            坂本龍一|                null|Mc1 (Live_2009_4_15)|
|user_000001|2009-05-04T13:38:31Z|a7f7df4a-77d8-4f1...|         

In [3]:
from pyspark.sql import SparkSession
# from pyspark.sql.functions import col  # Import the col function
import logging
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import unix_timestamp, lag, col
import pandas as pd
log = logging.getLogger("kedro")

In [25]:




#log.warning("Issue warning")
#log.info("Send information")


# Configure SparkSession with local mode and 6 worker threads
spark = SparkSession.builder.getOrCreate()
assert spark.conf.get("spark.app.name") == 'music-listening-behaviour'

# the aim is to group the playlist dataset by listening session 
# each song has been started within 15 minutes of the other 

# group by users and start with a user's minimum track session start time
# Define a new session ID everytime the time between consecutive tracks exceeds the treshold.
# Also define a global session ID, unique across users

tracksess = spark.read.csv('C:/GP/music-listening-behaviour/data/01_raw/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', sep = '\t').withColumnRenamed("_c0", "userid") \
                     .withColumnRenamed("_c1", "timestamp") \
                     .withColumnRenamed("_c2", "artistid") \
                     .withColumnRenamed("_c3", "artistname") \
                     .withColumnRenamed("_c4", "traid") \
                     .withColumnRenamed("_c5", "traname").orderBy(['userid', "timestamp"]) # catalog.load('track-sessions')
# tracksess.show(10)
#tracksess.count()



# defined in conf/base/parameters.yml
session_maxidletime = 900 # seconds 
long_session_quantile = 0.9
log.info('starting pipeline with session_maxidletime '+ str(session_maxidletime)+ ' long_session_quantile '+ str(long_session_quantile))

# define basic assertions
nb_distinct_users = tracksess.select("userid").distinct().count()
nb_distinct_users_profiles = tracksess.select("userid").distinct().count()
assert nb_distinct_users==992, "Unexpected number of users"

# Convert timestamp column to timestamp datatype
ts = tracksess.withColumn("timestamp", unix_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss'Z'").cast("timestamp"))

# Define a window specification for user-based operations
user_window = Window.partitionBy("userid").orderBy("timestamp")

# Calculate time differences between consecutive rows for each user
ts = ts.withColumn("time_diff", F.unix_timestamp("timestamp") - F.unix_timestamp(F.lag("timestamp").over(user_window))).orderBy(['userid', "timestamp"])

# Calculate session boundaries: a new session starts if time difference is greater than 15 minutes
# or if the time_diff is null (start of a new user's sessions)
ts = ts.withColumn("is_new_session", F.when((F.col("time_diff") > session_maxidletime) | (F.col("time_diff").isNull()), 1).otherwise(0))

# Calculate session ID for each session using cumulative sum of "is_new_session" column
ts = ts.withColumn("session_id", F.sum("is_new_session").over(user_window)).orderBy(['userid', "timestamp"])
#ts.show(10)

# Calculate session duration for each session
session_duration = ts.groupBy("userid", "session_id").agg(
    F.min("timestamp").alias("start_time"),
    F.max("timestamp").alias("end_time")
).withColumn("session_duration", F.unix_timestamp("end_time") - F.unix_timestamp("start_time")).orderBy(['userid', "session_id"])


# Calculate a unique session_id across all users
session_duration = session_duration.withColumn(
    "unique_session_id",
    F.row_number().over(Window.orderBy("userid", "session_id"))
)
#session_duration.show(10)
session_duration.count()

# Determine the threshold for a long session duration (top 10 percentile)
long_session_threshold = session_duration.approxQuantile("session_duration", [0.9], 0)[0] # error can be increased for performance
log.info('long session threshold ' + str(long_session_threshold))

# Identify users with at least one long session
users_with_long_sessions = session_duration.filter(F.col("session_duration") >= long_session_threshold).select("userid").distinct()
#users_with_long_sessions.show(100)

# assert that the quantile was correctly approximated
quantile_check = session_duration.agg(
    F.count(F.when(F.col("session_duration") < long_session_threshold, F.col("session_duration"))).alias("short_sessions_sum"),
    F.count(F.when(F.col("session_duration") >= long_session_threshold, F.col("session_duration"))).alias("long_sessions_sum")
)
qcheck = quantile_check.toPandas()
qcheck['short_long_ratio'] = qcheck["short_sessions_sum"] / (qcheck["short_sessions_sum"]+qcheck["long_sessions_sum"])
qcheck['long_short_ratio'] = qcheck["long_sessions_sum"] / (qcheck["short_sessions_sum"]+qcheck["long_sessions_sum"])
assert abs(qcheck.short_long_ratio[0]-long_session_quantile)<0.001, "threshold long session quantiles were not correctly defined"

# Count the total number of users and users with at least one long session
total_users = ts.select("userid").distinct().count()
users_with_long_session_count = users_with_long_sessions.count()
users_with_long_session_count

# Calculate the proportion of users with at least one long session
proportion_with_long_sessions = users_with_long_session_count / total_users

# Print the proportion
log.info("Proportion of users with at least one long session:" + str(proportion_with_long_sessions))


[08/29/23 21:21:18] INFO     starting pipeline with session_maxidletime 900 long_session_quantile    ]8;id=641119;file://C:\Users\gauth\AppData\Local\Temp\ipykernel_33152\65259357.py\65259357.py]8;;\:]8;id=175445;file://C:\Users\gauth\AppData\Local\Temp\ipykernel_33152\65259357.py#28\28]8;;\
                             0.9                                                                                   

[08/29/23 21:22:26] INFO     long session threshold 9025.0                                           ]8;id=921204;file://C:\Users\gauth\AppData\Local\Temp\ipykernel_33152\65259357.py\65259357.py]8;;\:]8;id=805890;file://C:\Users\gauth\AppData\Local\Temp\ipykernel_33152\65259357.py#68\68]8;;\

[08/29/23 21:23:26] INFO     Proportion of users with at least one long session:0.9485887096774194   ]8;id=597824;file://C:\Users\gauth\AppData\Local\Temp\ipykernel_33152\65259357.py\65259357.py]8;;\:]8;id=721754;file://C:\Users\gauth\AppData\Local\Temp\ipykernel_33152\65259357.py#93\93]8;;\

In [13]:
uids = catalog.load('userid-profiles')
uids.columns

[08/29/23 21:16:44] INFO     Loading data from 'userid-profiles' (CSVDataSet)...                data_catalog.py:475

Index(['#id', 'gender', 'age', 'country', 'registered'], dtype='object')

In [24]:
nb_distinct_users = tracksess.select("userid").distinct().count()
assert nb_distinct_users==len(uids['#id'].drop_duplicates())
log.info('nb_distinct_users '+str(nb_distinct_users))

[08/29/23 21:19:48] INFO     nb_distinct_users992                                                   ]8;id=640144;file://C:\Users\gauth\AppData\Local\Temp\ipykernel_33152\2169171363.py\2169171363.py]8;;\:]8;id=686961;file://C:\Users\gauth\AppData\Local\Temp\ipykernel_33152\2169171363.py#3\3]8;;\

In [28]:
# Convert timestamp column to timestamp datatype
ts = tracksess.withColumn("timestamp", unix_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss'Z'").cast("timestamp"))
ts.dtypes

[
    ('userid', 'string'),
    ('timestamp', 'timestamp'),
    ('artistid', 'string'),
    ('artistname', 'string'),
    ('traid', 'string'),
    ('traname', 'string')
]

In [33]:
dir(session)

[
    '__class__',
    '__delattr__',
    '__dict__',
    '__dir__',
    '__doc__',
    '__enter__',
    '__eq__',
    '__exit__',
    '__format__',
    '__ge__',
    '__getattribute__',
    '__gt__',
    '__hash__',
    '__init__',
    '__init_subclass__',
    '__le__',
    '__lt__',
    '__module__',
    '__ne__',
    '__new__',
    '__reduce__',
    '__reduce_ex__',
    '__repr__',
    '__setattr__',
    '__sizeof__',
    '__str__',
    '__subclasshook__',
    '__weakref__',
    '_conf_source',
    '_get_config_loader',
    '_get_logging_config',
    '_hook_manager',
    '_init_store',
    '_log_exception',
    '_logger',
    '_package_name',
    '_project_path',
    '_run_called',
    '_setup_logging',
    '_store',
    'close',
    'create',
    'load_context',
    'run',
    'save_on_close',
    'session_id',
    'store'
]

In [52]:
%reload_kedro

[08/29/23 22:53:02] INFO     Resolved project path as: C:\GP\music-listening-behaviour.             __init__.py:139
                             To set a different path, run '%reload_kedro <project_root>'                           

[08/29/23 22:53:03] INFO     Kedro project music-listening-behaviour                                __init__.py:108

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:109
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        __init__.py:115

In [5]:
session.run()

[08/29/23 23:40:20] INFO     Kedro project music-listening-behaviour                                 session.py:364

[08/29/23 23:40:21] INFO     Loading data from 'track_sessions' (SparkDataSet)...               data_catalog.py:475

[08/29/23 23:40:22] INFO     Loading data from 'userid_profiles' (CSVDataSet)...                data_catalog.py:475

                    INFO     Loading data from 'parameters' (MemoryDataset)...                  data_catalog.py:475

                    INFO     Running node: preprocess_format_tracksessions:                             node.py:331
                             preprocess_format_tracksessions([track_sessions,userid_profiles,parameters            
                             ]) -> [ts_formatted]                                                                  

[08/29/23 23:40:28] INFO     nb_distinct_users 992                                                      ]8;id=398545;file://C:\GP\music-listening-behaviour\src\music_listening_behaviour\pipelines\data_processing\nodes.py\nodes.py]8;;\:]8;id=351884;file://C:\GP\music-listening-behaviour\src\music_listening_behaviour\pipelines\data_processing\nodes.py#39\39]8;;\

                    INFO     Saving data to 'ts_formatted' (SparkDataSet)...                    data_catalog.py:514

[08/29/23 23:40:44] INFO     Completed 1 out of 1 tasks                                     sequential_runner.py:85

                    INFO     Pipeline execution completed successfully.                               runner.py:105

{}

In [7]:
profiles = catalog.load('userid_profiles')

[08/30/23 13:05:27] INFO     Loading data from 'userid_profiles' (CSVDataSet)...                data_catalog.py:475

In [8]:
profiles

,#id,gender,age,country,registered
0,user_000001,m,NaN,Japan,"Aug 13, 2006"
1,user_000002,f,NaN,Peru,"Feb 24, 2006"
2,user_000003,m,22.0,United States,"Oct 30, 2005"
3,user_000004,f,NaN,NaN,"Apr 26, 2006"
4,user_000005,m,NaN,Bulgaria,"Jun 29, 2006"
...,...,...,...,...,...
987,user_000996,f,NaN,United States,"Jul 17, 2006"
988,user_000997,m,NaN,United States,"Jan 5, 2007"
989,user_000998,m,NaN,United Kingdom,"Sep 28, 2005"
990,user_000999,f,NaN,Poland,"Jul 24, 2007"


In [ ]:
nb_distinct_users = tracksess.select("userid").distinct().count()

In [ ]:
nb_distinct_profiles = profiles.select("#id").distinct().count()

In [ ]:
assert nb_distinct_users == nb_distinct_profiles

In [ ]:
nb_distinct_users = tracksess.select("userid").distinct().count()
# nb_unique_users = 992
assert nb_distinct_users==len(userid_profiles['#id'].drop_duplicates())

In [9]:
session.run()

[08/30/23 13:19:09] INFO     Kedro project music-listening-behaviour                                 session.py:364

                    WARNING  C:\Program                                                             warnings.py:109
                             Files\Python310\lib\site-packages\kedro\framework\session\session.py:2                
                             66: FutureWarning: ConfigLoader will be deprecated in Kedro 0.19.                     
                             Please use the OmegaConfigLoader instead. To consult the documentation                
                             for OmegaConfigLoader, see here:                                                      
                             https://docs.kedro.org/en/stable/configuration/advanced_configuration.                
                             html#omegaconfigloader                                                                
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Loading data from 'track_sessions' (SparkDataSet)...               data_catalog.py:475

[08/30/23 13:19:11] INFO     Loading data from 'userid_profiles' (CSVDataSet)...                data_catalog.py:475

                    INFO     Loading data from 'parameters' (MemoryDataset)...                  data_catalog.py:475

...

[08/30/23 13:20:49] INFO        users_with_long_session_count  total_users                             ]8;id=166354;file://C:\GP\music-listening-behaviour\src\music_listening_behaviour\pipelines\data_processing\nodes.py\nodes.py]8;;\:]8;id=216351;file://C:\GP\music-listening-behaviour\src\music_listening_behaviour\pipelines\data_processing\nodes.py#127\127]8;;\
                             proportion_with_long_sessions  \                                                      
                             0                            941          992                                         
                             0.948589                                                                              
                                                                                                                   
                                long_session_threshold  total short sessions  total long sessions  \               
                             0                  9025.0               1059361               117715                  
                                                                                                                   
                                long_session_ratio                                                                 
                             0            0.100006                                                                 

                    INFO     Saving data to 'results_summary' (ExcelDataSet)...                 data_catalog.py:514

                    INFO     Completed 2 out of 2 tasks                                     sequential_runner.py:85

                    INFO     Pipeline execution completed successfully.                               runner.py:105

{}